In [ ]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import r2_score
import dask_ml
import dask
import pandas as pd
import numpy as np
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*Sending large graph.*")

from dask.distributed import Client, LocalCluster
import dask.multiprocessing

cluster = LocalCluster(processes=True,n_workers=6, threads_per_worker=1)
client = Client(cluster)
import sys
import pickle 

from data_helpers import *
from metrics import *

In [ ]:
folders = [
    #'train0_25',
    #'train25_50',
    'train50_75',
    #'train75_100'
]

# Read Parquet files from each folder into Dask DataFrames
dfs = [dd.read_parquet(folder) for folder in folders]

# Concatenate all DataFrames into a single DataFrame
data = dd.concat(dfs)

In [ ]:
np.random.seed(42)

orig_partitions = [i for i in range(0,int(data.npartitions))]
np.random.shuffle(orig_partitions) #shuffles inplace

trainSep = int(0.95* data.npartitions)
valEnd = data.npartitions #int(0.05* data.npartitions) + trainSep

sampledPartIdxTrain = orig_partitions[0:trainSep]
sampledPartIdxTest  = orig_partitions[trainSep:valEnd]

In [ ]:
# validation data
partPerLoop = 35

for i in range(1):
    startPartIdx = i*partPerLoop
    val, combinedF,transT = concatData(data, partPerLoop, startPartIdx, sampledPartIdxTest)

In [ ]:
def concatSampledData(data, partPerLoop, startPartIdx,sampledPartIdx, f, mean, separator):
    dfLarge = []
    dfSmall = []
    for j in range(partPerLoop):
        a = data.get_partition(int(sampledPartIdx[startPartIdx+j])).compute()
        a, newF = addFeatures(a)

        tr_large = a.loc[abs(a[f]) > separator*abs(mean)]
        tr_small = a.loc[abs(a[f]) < separator*abs(mean)]

        tr_small = tr_small.sample(n=1000, random_state=42)

        
        dfLarge.append(tr_large)
        dfSmall.append(tr_small)
        allF = features60+newF+feat1
    
    return pd.concat(dfLarge), pd.concat(dfSmall), allF

In [ ]:
# training sequentially
partPerLoop = len(sampledPartIdxTrain)

f = 'ptend_q0002_26'
mean_f = data[f].mean().compute()

for i in range(1):
    startPartIdx = i*partPerLoop
    tr_large, tr_small, allF = concatSampledData(data, partPerLoop, startPartIdx, sampledPartIdxTrain, f, mean_f, separator=3)  # sep = 10 -> only like 20 large vals

In [ ]:
std = data[f].std().compute()

In [ ]:
mean_f,std

In [ ]:
import sys
sys.getsizeof(tr_large)/1e6, sys.getsizeof(tr_small)/1e6 #in mb

In [ ]:
tr_large